In [ ]:
from __future__ import print_function
import os,sys
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
from readparquet import UBParquetReader
from detectoroutline import DetectorOutline

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg+other
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255)), # proton
                               dtype=np.float ) 

In [ ]:
# DATA FILES
datafile="temp.parquet"

reader = UBParquetReader(datafile)
NENTRIES = reader.nentries

In [ ]:
# Get entry data
ENTRY = 0
data = reader.get_entry(ENTRY)

In [ ]:
# make spacepoint visualization with particle type labels
no_ghost_points = True
pos = data['spacepoint_t'][:,:3]
label = data['ssnet_label']
#print(label.shape,label.dtype)
color = np.zeros( (pos.shape[0],3), dtype=np.float )
for i in range(6):
   color[ label==i ] = ssnet_color_array[i,:]

if no_ghost_points:
    pos = pos[data['matchtriplet'][:,3]==1]
    color = color[data['matchtriplet'][:,3]==1]
print(pos.shape)
                 
plot_ssnet = {"type":"scatter3d",
              "x": pos[:,0],
              "y": pos[:,1],
              "z": pos[:,2],
              "mode":"markers",
              "name":"particletype",
              "marker":{"color":color,"size":1,"opacity":0.8}}


# PLOTLY
detlines = detdata.getlines(color=(10,10,10))

# DATA
plotdata = [plot_ssnet] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Particle-type Labels, Spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 1, "y": 1, "z": 1},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plotdata, layout=layout)
fig.show()

In [ ]:
# U-PLANE
img_u = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(0) 
plot_uplane = [ lardly.data.visualize_larcv_image2d( img_u ) ]

fig = go.Figure(data=plot_uplane)
fig.show()

In [ ]:
# V-PLANE
img_v = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(1) 
plot_vplane = [ lardly.data.visualize_larcv_image2d( img_v ) ]

fig = go.Figure(data=plot_vplane)
fig.show()

In [ ]:
# Y-PLANE
img_y = iolcv.get_data(larcv.kProductImage2D,"wire").as_vector().at(2) 
plot_yplane = [ lardly.data.visualize_larcv_image2d( img_y ) ]

fig = go.Figure(data=plot_yplane)
fig.show()

In [ ]:
# Truth Metadata
print "Enu: %.2f MeV"%(fvv.Enu_1e1p)
print "BDT: %.4f"%(fvv.BDTscore_1e1p)
print "showerQ=(%.2f,%.2f,%.2f)"%(fvv.shower1_sumQ_U, fvv.shower1_sumQ_V, fvv.shower1_sumQ_Y)